In [119]:
from bs4 import BeautifulSoup
import requests
import numpy as np
import pandas as pd
from collections import OrderedDict, defaultdict

In [ ]:
## Looop over pages 2-99
## Loop over all plugins on page
## Extract data into one dictionary
## Turn dictionary into dataframe

In [195]:
def get_star_count(plugin_soup):
    full = len(re.findall('dashicons-star-filled',str(plugin_soup)))
    half = len(re.findall('dashicons-star-half',str(plugin_soup)))
    return full + (0.5*half)

In [130]:
#for p in range(2,100):
 #   print(p)

In [196]:
plugin_infos = []

In [4]:
result = requests.get("https://de.wordpress.org/plugins/browse/popular/page/2/")

In [5]:
result.status_code

200

In [6]:
c = result.content

In [52]:
soup = BeautifulSoup(c, 'html.parser')

In [131]:
plugins =  soup.find_all('article')

In [203]:
for plugin in plugins:
    print(plugin.find('header', {'class' : 'entry-header'}).get_text()[1:-1])
    plugin_info = {}
    plugin_info['plugin_name'] = plugin.find('header', {'class' : 'entry-header'}).get_text()[1:-1]
    #plugin_info['author'] = plugin.find('span', {'class' : 'plugin-author'}).get_text()[2:-2]
    plugin_info['rating_count'] = plugin.find('div', {'class':'plugin-rating'}).get_text()[1:-1].split(" ")[0]
    plugin_info['rating_stars'] = get_star_count(plugin)
    #plugin_info['active_installs'] = plugin.find('span', {'class' : 'active-installs'}).get_text()[5:-2]
    #plugin_info['tested_with'] = plugin.find('span', {'class' : 'tested-with'}).get_text()[6:-3]
    #plugin_info['last_updated'] = plugin.find('span', {'class' : 'last-updated'}).get_text()
    plugin_info['description'] = plugin.find('div', {'class' : 'entry-excerpt'}).get_text()[1:-1]
    plugin_infos.append(plugin_info)

UpdraftPlus WordPress Backup Plugin
Regenerate Thumbnails
Page Builder by SiteOrigin
Advanced Custom Fields
Google Analytics Dashboard for WP (GADWP)
Hello Dolly
W3 Total Cache
All-in-One WP Migration
Duplicator – WordPress Migration Plugin
MailChimp for WordPress
SiteOrigin Widgets Bundle
Smush Image Compression and Optimization
Ninja Forms – The Easy and Powerful Forms Builder
Contact Form by WPForms – Drag & Drop Form Builder for WordPress


In [205]:
pd.DataFrame(plugin_infos)

,description,plugin_name,rating_count,rating_stars
0,Backup und Wiederherstellung leicht gemacht. V...,UpdraftPlus WordPress Backup Plugin,2.830,5.0
1,Regenerate the thumbnails for one or more of y...,Regenerate Thumbnails,314,4.5
2,"Erstelle responsive Seitenlayouts mit Widgets,...",Page Builder by SiteOrigin,934,5.0
3,"Erweitere WordPress um vielseitige, profession...",Advanced Custom Fields,1.025,5.0
4,Connects Google Analytics with your WordPress ...,Google Analytics Dashboard for WP (GADWP),359,4.5
5,"Das ist nicht nur ein Plugin, sondern symbolis...",Hello Dolly,196,2.5
6,Search Engine (SEO) & Performance Optimization...,W3 Total Cache,4.037,4.5
7,"Move, transfer, copy, migrate, and backup a si...",All-in-One WP Migration,4.364,5.0
8,WordPress-Migration und Backups sind so viel e...,Duplicator – WordPress Migration Plugin,1.703,5.0
9,"MailChimp for WordPress, the #1 MailChimp plugin.",MailChimp for WordPress,975,5.0


In [185]:
plugins[0].find('div', {'class':'plugin-rating'}).get_text()[1:-1].split(" ")[0]

'2.830'

In [160]:
plugins[0].find('span', {'class' : 'dashicons dashicons-star-filled'}).get_text()[1:-1]

''

In [ ]:
Plugin Name

###Plugin Name

In [137]:
plugins[0].find('header', {'class' : 'entry-header'}).get_text()[1:-1]

'UpdraftPlus WordPress Backup Plugin'

In [153]:
plugins[0].find('div', {'class' : 'entry-excerpt'}).get_text()[1:-1]

'Backup und Wiederherstellung leicht gemacht. Vollständige Backups: manuell oder geplant (Sicherung in Dropbox, S3, Google…'

In [148]:
articles[0].find('span', {'class' : 'plugin-author'}).get_text()[2:-2]

'UpdraftPlus.Com, DavidAnderson'

In [187]:
articles[0].find('span', {'class' : 'active-installs'}).get_text()[5:-2]

'1+ Millionen aktive Installationen'

In [190]:
articles[0].find('span', {'class' : 'tested-with'}).get_text()[6:-3]

'Getestet mit 4.9.6'

In [192]:
articles[0].find('span', {'class' : 'last-updated'}).get_text()[6:-2]

'Zuletzt aktualisiert vor 4 Wochen'